SOME FILES (IF DOING SIMULTANEOUS LOWER BAND OR MULTI-CONSTELLATION L1) WILL WANT TO USE THE SAME SIGNAL BUFFER BUT WITH DIFFERENT CENTER FREQUENCIES.

### signal data sources

The signal data, which was at some point generated by a radio front-end, can come from a number of different places.

The first is the file source, which is most likely used in post-processing context.

For signals read from file, we want to abstract the different file types. Different file types may have different ways of storing data. Because the relative number of actual file types used is small compared to the number of combinations of file type parameters, we should define a FileType child class for each file type.

File type objects should include functionality to read a given number of samples from a file into either a real or complex buffer.

Some files have headers in them, possibly at the beginning, or every 1ms, or some other interval? We need to remove these headers from the data (and sometimes use the header data). To do this, we make a `Header` class that encapsulates and abstracts removing/parsing headers.

In [ ]:
class FileType:
    '''
    Abstracts the information about headers in files.
    '''
    
    def __init__(self, size, period):
        self.size = size
        self.period = period
    
    def remove_header(self, buffer, file_offset):
        '''
        `buffer` should be a file byte buffer.
        `file_offset` is the offset of the buffer into the file.
        '''
        return buffer

##### buffer size

Strategy:
Set buffer size to be quite large. Guarantee a minimum overlap between updates of the buffer. All algorithms should be able to run within this minimum overlap. Fine acquisition might use up to 100ms. If 100ms is the maximum alloted space for algorithms to run, then the buffer should maybe be 3 times that: 300ms * f_samp = 12 MB buffer--this is not at all small...

In [2]:
%%writefile ../../gnss/receiver/sources.py

# block should return block of data of requested size and the block epoch

from os.path import getsize
from os import SEEK_SET
from numpy import zeros, byte, fromfile, uint8, int8, bitwise_and


class SignalSource:
    """
    Provides an abstract interface to digitized data from a GNSS front-end.
    The actual source might be a file, in-memory buffer from USRP, or the network.
    These interfaces should be implemented as derived classes of `SignalSource`.
    
    We can send the Channels sliced views of the buffer. These views do not copy by default.
    
    `decimate` is by how many samples the source should decimate the signal
    """
    
    def __init__(self, f_samp, f_center, buffer_size, bit_depth=8, real=False, decimate=None):
        self.f_samp = f_samp
        self.f_center = f_center
        self.bit_depth = bit_depth
        self.real = real
        self.decimate = decimate
        self.buffer_size = buffer_size
        data_type = float if self.real else complex
        self.buffer = zeros((buffer_size,), dtype=data_type)
        self.buffer_start_time = 0.
        
    # abstract???
    def get(self, block_size, time=None):
        """
        If `time` is None, returns buffer start time with block at beginning of buffer
        """
        if time:
            delta = time - self.buffer_start_time
            n = round(delta * self.f_samp)
            if n < 0 or len(self.buffer) <= n:
                raise Exception('time outside of buffer range')
        else:
            n = 0
        if len(self.buffer) <= n + block_size:
            raise Exception('requested sample range extends beyond buffer sample range')
        time = n / self.f_samp
        return self.buffer[n:n + block_size], time
    
    def min_time(self):
        return self.buffer_start_time
    
    def max_time(self):
        return self.buffer_start_time + self.buffer_size / self.f_samp


class FileSignalSource(SignalSource):
    """
    Signal source that reads signal data from a file.
    
    `file_loc` is the location of the next unread btye from the file, i.e.
    the subsequent sample to the last sample in the current buffer
    
    `header` describes headers (right now they will just be removed). If 
    `None` (default) ignores headers.
    """
    
    MAX_BUFFER_SIZE = 100000000  # 100 MSamples
    
    def __init__(self, filepath, f_samp, f_center, buffer_size=None, bit_depth=8, real=True, decimate=False, header=None):
        self.filepath = filepath
        self.file_loc = 0
        self.header = header
        self.file_size = min(getsize(filepath), MAX_BUFFER_SIZE)
        if not buffer_size:
            buffer_size = self.file_size
        super(FileSignalSource, self).__init__(f_samp, f_center, buffer_size, bit_depth, real)
    
    def load(self, overlap=0):
        '''
        Loads data from file into SignalSource buffer.
        Curent supported data formats:
        8-bit real
        4-bit complex w/ upper nibble real
        '''
        if overlap:
            self.buffer[:overlap] = self.buffer[-overlap:]
        with open(self.filepath, "rb") as f:  # reopen the file
            f.seek(self.file_loc, SEEK_SET)   # seek
            # TODO handle different bit depths and complex types
            if self.real and self.bit_depth == 8:
                temp = fromfile(f, dtype=byte, count=int(self.buffer_size - overlap))
                if self.header:
                    temp = header.remove(temp, self.file_loc)
                self.buffer[:] = 
            elif not self.real and self.bit_depth == 4:
                temp = fromfile(f, dtype=byte, count=int(self.buffer_size - overlap))
                if self.header:
                    temp = header.remove(temp, self.file_loc)
                # typically real is upper nibble, complex is lower nibble, but TODO make this generic
#                 real = (temp & 0x0F).astype(int8)
#                 imag = ((temp & 0xF0) >> 4).astype(int8)
#                 real[real & 0b1000 > 0] -= 2**4
#                 imag[imag & 0b1000 > 0] -= 2**4
                real = (bitwise_and(temp, 0x0f) << 4).astype(int8) >> 4
                imag = bitwise_and(temp, 0xf0).astype(int8) >> 4
                self.buffer[:] = real + 1j * imag
            # if it is Steve data at 100MHz, remove the header
            
    def advance(self, overlap=100000):
        # TODO handle overlap and buffer size incompatibilities?
        self.file_loc += self.buffer_size - overlap
        self.load(overlap)
        

Overwriting ../../gnss/receiver/sources.py


Test for complex 4-bit data parsing:

In [1]:
import numpy as np

In [24]:
temp = np.zeros((8,), dtype=np.byte)
temp[:2] = 0x0F
temp[2:4] = 0x07
temp[4:6] = 0xD0
temp[6:] = 0x50

real = (temp & 0x0F).astype(np.int8)
imag = ((temp & 0xF0) >> 4).astype(np.int8)
real[real & 0b1000 > 0] -= 2**4
imag[imag & 0b1000 > 0] -= 2**4
out = real + 1j * imag

print(temp, out)

[ 15  15   7   7 -48 -48  80  80] [-1.+0.j -1.+0.j  7.+0.j  7.+0.j  0.-3.j  0.-3.j  0.+5.j  0.+5.j]


In [ ]:
f_center = 0.
f_samp = 5e6
filepath = '/mnt/gluster/by-location/ascension-island/a1/gpsl2/usrp5/20130307_184310'
source = sources.FileSignalSource(filepath, f_samp=f_samp, f_center=f_center, buffer_size=10e6, bit_depth=4, real=False)
source.load()
print('we have {0} seconds of data'.format(source.buffer_size / source.f_samp))
data = source.buffer[:2000]
fig = figure(title='hist')
hist = Histogram(values=np.concatenate((np.real(data), np.imag(data))), bins=32)
show(hist)